<a href="https://colab.research.google.com/github/lee-messi/machine-learning/blob/main/cola_grammar_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow --quiet
!pip install tensorflow-hub --quiet
!pip install tensorflow-text --quiet
!pip install tensorflow-addons --quiet
!pip install tensorflow-datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 44.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.6.11 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.2.6 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.1 MB/s eta 0:00:00


In [73]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [74]:
if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

Using TPU


## Import CoLA (The Corpus of Linguistic Acceptability) Data

CoLA is a collection of sentences that has been annotated for acceptability. The sentences are coded/labeled 1 if the sentence is grammatically acceptable or correct, and they are coded 0 if the sentence is grammatically wrong. You can access the publicly available corpus using the link [here](https://nyu-mll.github.io/CoLA/). First, we are going to import the datasets. The datasets are provided in **.tsv** format which can be imported using the **read_csv()** function.

In [80]:
train = pd.read_csv('drive/MyDrive/Colab Notebooks/glue-tasks/in_domain_train.tsv',
                    sep = '\t', header = None)
val = pd.read_csv('drive/MyDrive/Colab Notebooks/glue-tasks/in_domain_dev.tsv',
                  sep = '\t', header = None)
test = pd.read_csv('drive/MyDrive/Colab Notebooks/glue-tasks/out_of_domain_dev.tsv',
                   sep = '\t', header = None)

In [81]:
train.columns = ['misc', 'labels', 'na', 'text']
val.columns = ['misc', 'labels', 'na', 'text']
test.columns = ['misc', 'labels', 'na', 'text']

In [82]:
'{}{}{}'.format(train.shape, val.shape, test.shape)

'(8551, 4)(527, 4)(516, 4)'

First, we are going to inspect the training dataset to ensure that it is balanced - the number of sentences that are grammatically correct and wrong are equal. This is so that the classifier is not biased.

In [83]:
train.labels.value_counts()

1    6023
0    2528
Name: labels, dtype: int64

There are 6,023 sentences with label 1 and 2,528 sentences with label 0. Let's balance these out.

In [78]:
train = train.groupby('labels').sample(1000)

Then, we are going to implement the **df_to_dataset()** function to create a **tf.data.Dataset** using the balanced reviews dataset. This allows us to map the features in the pandas dataframe to features that are more appropriate for training. You can read more about this and check out the function that is used to perform this task [here](https://www.tensorflow.org/tutorials/structured_data/feature_columns). Then, we are going to map the training, validation, and test datasets using the function. Note that depending on the features that you use in the model, you may have to modify parts of the function.

In [67]:
def df_to_dataset(dataframe, shuffle = True, batch_size = 128):
    df = dataframe.copy()
    labels = df.labels
    df = df.text
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    if shuffle == True:
        ds = ds.shuffle(buffer_size = len(df))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return(ds)

In [68]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val)
test_ds = df_to_dataset(test)

## Classifier Model using BERT

In [84]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [63]:
bert_preprocess = hub.KerasLayer(tfhub_handle_preprocess)
bert_encoder = hub.KerasLayer(tfhub_handle_encoder)

In [69]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string)
encoder_input = bert_preprocess(text_input)
encoder_output = bert_encoder(encoder_input)

l = tf.keras.layers.Dense(100, activation = 'relu')(encoder_output['pooled_output'])
l = tf.keras.layers.Dense(1, activation = 'sigmoid')(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [70]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [71]:
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = 10)

Epoch 1/10
16/16 [==============================] - 69s 4s/step - loss: 1.0159 - accuracy: 0.5390 - val_loss: 0.6686 - val_accuracy: 0.6528
Epoch 2/10
16/16 [==============================] - 64s 4s/step - loss: 0.6857 - accuracy: 0.5655 - val_loss: 0.6824 - val_accuracy: 0.6490
Epoch 3/10
16/16 [==============================] - 64s 4s/step - loss: 0.6793 - accuracy: 0.5755 - val_loss: 0.6714 - val_accuracy: 0.6186
Epoch 4/10
16/16 [==============================] - 64s 4s/step - loss: 0.6708 - accuracy: 0.5970 - val_loss: 0.6359 - val_accuracy: 0.6546
Epoch 5/10
16/16 [==============================] - 64s 4s/step - loss: 0.6675 - accuracy: 0.5925 - val_loss: 0.6529 - val_accuracy: 0.6205
Epoch 6/10
16/16 [==============================] - 64s 4s/step - loss: 0.6642 - accuracy: 0.5980 - val_loss: 0.6630 - val_accuracy: 0.6110
Epoch 7/10
16/16 [==============================] - 64s 4s/step - loss: 0.6498 - accuracy: 0.6245 - val_loss: 0.6128 - val_accuracy: 0.6812
Epoch 8/10
16/16 [==

## Evaluate Model

In [85]:
model.predict(['he good person is a monster in the woods.'])

1/1 [==============================] - 0s 248ms/step


array([[0.36027667]], dtype=float32)

We are going to test out the model using an example sentence that is grammatically wrong. As the prediction value is smaller than 0.5, the model predicted that the sentence is grammatically wrong. This time, we are going to evaluate the model using the test dataset:

In [86]:
model.evaluate(test_ds)

5/5 [==============================] - 14s 3s/step - loss: 0.6436 - accuracy: 0.6395


[0.6435754895210266, 0.6395348906517029]

The model returned an accuracy of **63.95%**. The model ranks around 15th among the models that have been used to perform the identical task on [Kaggle](https://www.kaggle.com/competitions/cola-in-domain-open-evaluation/leaderboard) (although the leaderboard is quite outdated).